In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime
import os
import sys
from selenium import webdriver
import json
import requests
import time

In [2]:
def conseguir_categorias():
    url="https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001"

    driver = webdriver.Chrome()
    pag=urllib.request.urlopen(url)
    driver.get(url)
    pag=driver.page_source
    doc=BeautifulSoup(pag,'html.parser')
    url_l=[]
    driver.close()
    for i in doc.find_all("li",attrs={'data-test-id':'categories-list-element'})[1:]:
        driver = webdriver.Chrome()
        pag=urllib.request.urlopen(url)
        driver.get(url)
        pag=driver.page_source
        doc=BeautifulSoup(pag,'html.parser')
        
        print(url)
        
        for j in doc.find_all("div",class_="sub-category-item"):
            
            sec=j.find("a").get("href")
            
            print(sec)
            #url_l.append("https://www.dia.es"+sec)
            url_l.append(sec)
        
        driver.close()
        url="https://www.dia.es"+i.find("a").get("href")
    driver = webdriver.Chrome()
    pag=urllib.request.urlopen(url)
    driver.get(url)
    pag=driver.page_source
    doc=BeautifulSoup(pag,'html.parser')

    print(url)
    for j in doc.find_all("div",class_="sub-category-item"):
        
        sec=j.find("a").get("href")
        
        print(sec)
        #url_l.append("https://www.dia.es"+sec)
        url_l.append(sec)
    driver.close()
    return url_l

In [3]:
enlace=conseguir_categorias()

https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001
/charcuteria-y-quesos/c/L101
/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001
/charcuteria-y-quesos/jamon-curado-y-paleta/c/L2004
/charcuteria-y-quesos/lomo-chorizo-fuet-salchichon/c/L2005
/charcuteria-y-quesos/queso-curado-semicurado-y-tierno/c/L2007
/charcuteria-y-quesos/queso-fresco/c/L2008
/charcuteria-y-quesos/queso-azul-y-roquefort/c/L2009
/charcuteria-y-quesos/quesos-fundidos-y-cremas/c/L2010
/charcuteria-y-quesos/quesos-internacionales/c/L2011
/charcuteria-y-quesos/salchichas/c/L2206
/charcuteria-y-quesos/foie-pate-y-sobrasada/c/L2012
https://www.dia.es/carniceria/c/L102
/carniceria/c/L102
/carniceria/pollo/c/L2202
/carniceria/vacuno/c/L2013
/carniceria/cerdo/c/L2014
/carniceria/pavo/c/L2015
/carniceria/conejo/c/L2016
/carniceria/hamburguesas-y-carne-picada/c/L2017
https://www.dia.es/pescados-mariscos-y-ahumados/c/L103
/pescados-mariscos-y-ahumados/c/L103
/pescados-maris

In [4]:
for k_idx,k in enumerate(enlace):
   

    
    nombre=[]
    categoria=[]
    subcategoria=[]
    peso=[]
    precio=[]
    precio_unidad=[]
    unidades=[]
    comercio=[]
    fecha=[]
    try:
        url="https://www.dia.es/api/v1/plp-back/reduced"+enlace[k_idx]
        print(url)
        pag=urllib.request.urlopen(url)
        doc=BeautifulSoup(pag)
        resp = requests.get(url=url)
        data=resp.json()
        for i in range(len(data["plp_items"])):
            nombre.append(data["plp_items"][i]["display_name"])
            categoria.append(url.split("/")[7])
            subcategoria.append(url.split("/")[8])
            peso.append(data["plp_items"][i]["weight"])
            precio.append(data["plp_items"][i]["prices"]["price"])
            precio_unidad.append(data["plp_items"][i]["prices"]["price_per_unit"])
            unidades=data["plp_items"][i]["prices"]["measure_unit"]
            comercio.append("dia")
            fecha.append(str(datetime.today()).split(" ")[0])
    except:
        print("error",enlace[k_idx])
        pass

    
    pag_nav=2
    while True:
        try:
            if len(data["plp_items"])>=20:
                new_url="https://www.dia.es/api/v1/plp-back/reduced"+"/".join(enlace[k_idx].split("/")[:-2])+f"/pag-{pag_nav}/"+"/".join(enlace[k_idx].split("/")[-2:])
                pag=urllib.request.urlopen(new_url)
                doc=BeautifulSoup(pag)
                resp = requests.get(url=new_url)
                data=resp.json()
                print(new_url)

                for i in range(len(data["plp_items"])):
                    nombre.append(data["plp_items"][i]["display_name"])
                    categoria.append(url.split("/")[7])
                    subcategoria.append(url.split("/")[8])
                    peso.append(data["plp_items"][i]["weight"])
                    precio.append(data["plp_items"][i]["prices"]["price"])
                    precio_unidad.append(data["plp_items"][i]["prices"]["price_per_unit"])
                    unidades=data["plp_items"][i]["prices"]["measure_unit"]
                    comercio.append("Dia")
                    fecha.append(str(datetime.today()).split(" ")[0])
                    
                pag_nav=pag_nav+1
            else:
                break
        except:
            break
        
    
    new_df=pd.DataFrame(data={
             "nombre":nombre,
             "categoria":categoria,
             "subcategoria":subcategoria,
             "peso":peso,
             "precio":precio,
             "precio_unidad":precio_unidad,
             "unidades":unidades,
             "comercio":comercio,
             "fecha":fecha           

        })
        
    if os.path.isfile("dia.csv")==True:
        datos=pd.read_csv("dia.csv",sep="|",index_col=False)
        final_df=pd.concat([datos,new_df],axis=0)
        final_df.to_csv("dia.csv",sep="|",index=False,encoding="utf-8")
    else:
            new_df.to_csv("dia.csv",sep="|",index=False,encoding="utf-8")  




https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/c/L101
error /charcuteria-y-quesos/c/L101
https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001
https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/pag-2/c/L2001
https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/pag-3/c/L2001
https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/jamon-curado-y-paleta/c/L2004
https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/jamon-curado-y-paleta/pag-2/c/L2004
https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/lomo-chorizo-fuet-salchichon/c/L2005
https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/lomo-chorizo-fuet-salchichon/pag-2/c/L2005
https://www.dia.es/api/v1/plp-back/reduced/charcuteria-y-quesos/lomo-chorizo-fuet-salchichon/pag-3/c/L2005
https://www.dia.es/api/v1/plp-back/re